### GROUP 21


## ADM_HOMEWORK 3

####   *  ATEFE MORADAN
####   *  GIUSEPPE STEFANALLI
####   *  VIKRANTH ALE


######  Importing required modules to build [*airbnb*](https://www.airbnb.com/) Search Engine 

In [23]:
import pandas as pd
import numpy as np
import csv
import json
import string
import unicodedata
import re
import folium
import geopy
import geopy.distance
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from scipy.spatial.distance import cosine
from geopy.geocoders import Nominatim
from geopy.distance import great_circle
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

### Step 1 :
######  Loading data into a DataFrame for Analysis

In [25]:
df=pd.read_csv("/home/atefe/Desktop/airbnb-property-data-from-texas/Airbnb_Texas_Rentals.csv")
df=df.dropna(axis=0, subset=['description','title'])
df.to_csv("/home/atefe/Desktop/airbnb-property-data-from-texas/Airbnb_Texas_Rentalsd.csv")

### Step 2 :
###### Reading the CSV file and writing the data into TSV files
######        Creating  .tsv file for each record of the dataset into a new Document for corresponding Doc_Id's

In [6]:
with open("/home/atefe/Desktop/airbnb-property-data-from-texas/Airbnb_Texas_Rentalsd.csv","r",encoding="utf8") as csvfile:
    t=csv.reader(csvfile)
    with open("/home/atefe/Desktop/airbnb-property-data-from-texas1/Airbnb_Texas_Rentals1.tsv","w",encoding="utf8") as tsvfile:
        t1=csv.writer(tsvfile,delimiter="\t")
        for line in t:
            t1.writerow(line)

In [7]:
with open("/home/atefe/Desktop/airbnb-property-data-from-texas1/Airbnb_Texas_Rentals1.tsv","r",encoding="utf8") as csvfile:
    t=csv.reader(csvfile)
    i=0
    for line in t:
        if i!=0:
            line=' '.join(line)
            line=line.split('\t')
            with open("/home/atefe/Desktop/airbnb-property-data-from-texas4/doc_"+str(i)+".tsv","w",encoding="utf8") as tsvfile:
                csv.writer(tsvfile,delimiter="\t").writerow(line[1:])
        i+=1 

### Step 3 :
#####  Defining Functions to clean the data for Removing Punctuation, Accents and to Create Stem Sentences inside the DataFrame

In [40]:
def stem_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

In [41]:
o=string.punctuation+'“–'
def remove_punctuations(text):
    for punctuation in o:
        text = text.replace(punctuation, '')
    return text

In [42]:
def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

In [180]:
stemmer = SnowballStemmer("english")
df=pd.DataFrame(columns=['title','description'])
ps = PorterStemmer()
stop = stopwords.words('english')
l = ['average_rate_per_night', 'bedrooms_count', 'city', 'date_of_listing', 'description' , 'latitude', 'longitude','title','url']
for i in range(1,18255):
    d=pd.read_csv("/home/atefe/Desktop/airbnb-property-data-from-texas4/doc_"+str(i)+".tsv",names=l,delimiter="\t")
    d['title']=d['title'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    d['description']=d['description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    d['title'] = d['title'].apply(lambda x:''.join([i for i in x 
                                                  if i not in string.punctuation]))
    d['description'] = d['description'].apply(lambda x:''.join([i for i in x 
                                                  if i not in string.punctuation]))
    d['title'] = d['title'].apply(stem_sentences)
    d['description'] = d['description'].apply(stem_sentences)
    d['title'] = d['title'].str.replace('\d+', '')
    d['description'] = d['description'].str.replace('\d+','')
    d['title'] = d['title'].apply(remove_punctuations)
    d['description'] = d['description'].apply(remove_punctuations)
    d['title'] = d['title'].apply(remove_accents)
    d['description'] = d['description'].apply(remove_accents)
    df=df.append(d[['title','description']])
    d.to_csv("/home/atefe/Desktop/airbnb-property-data-from-texas5/doc_"+str(i)+".tsv",header=False,index=False,sep="\t")
    d[['title','description']].to_csv("/home/atefe/Desktop/airbnb-property-data-from-texas6/docI_"+str(i)+".tsv",header=False,index=False,sep="\t")

In [181]:
# Saving the Data into a CSV file after Cleaning it
#This is done to minimise time and efforts in carrying out above repeatative steps 2 - 4

df.to_csv("new_backup_title_desc.csv")

### Step 3.1 :
######  Performing Text Analysis for the Columns ' Title ' & ' Description '  

In [26]:
df=pd.read_csv("new_backup_title_desc.csv")
#As we decided to merge these two columns and if one column is NAN , if we add them together we will get the NAN
#We decided to fill the na values

In [27]:
df.reset_index(drop=True)

,Unnamed: 0,title,description
0,1,privat roomsbathroom min iah airport,welcom stay privat room queen bed detach priva...
1,2,uniqu locat alamo height design inspir,stylish fulli remodel home upscal nw alamo he...
2,3,river hous near citi,river hous island close citi na well maintain ...
3,4,privat room close campus,privat bedroom cute littl home situat covet ga...
4,5,the porch,welcom origin s home we recent purchas home m...
5,6,gorgeous home countri set,my place close lake conro familyfriend activ n...
6,7,cozi quaint countri retreat acreag,rustic countri retreat acr southeast austin c...
7,8,friend privat room quiet neighborhood,this beauti bedroom queen size bed closet we p...
8,9,bayfront condo,first class comfort condo best view bottom dec...
9,10,cozi histor privat studio,privat entranc


In [28]:
df.to_csv('vocabulary.txt', header=None, index=None,sep='\t')

In [29]:
f = open('vocabulary.txt', 'r+')
lines = [line for line in f.readlines()]
f.close()

In [125]:
vectorizer = CountVectorizer()
vectorizer.fit(lines)
vectorize=TfidfVectorizer()
X = vectorize.fit_transform(lines)

In [31]:
termid=vectorizer.vocabulary_

In [32]:
import json
def default(o):
    if isinstance(o, np.int64): return int(o)  
    raise TypeError

In [33]:
with open('termiddic.json', 'w') as fp:
    json.dump(termid, fp, default=default)

In [195]:
j=[]
for i in range(1,18255):
    f=open("/home/atefe/Desktop/airbnb-property-data-from-texas6/docI_"+str(i)+".tsv", "r")
    text=f.read()
    vectorizer = CountVectorizer()
    X1 = vectorizer.fit_transform(text.split())
    l1=set(vectorizer.get_feature_names())
    j.append(l1)

#### Step 3.1.1 :
######  Creating an Inverted Index to build a Search Engine

In [196]:
from collections import defaultdict
inv_indx = defaultdict(list)
for idx, text in enumerate(j):
    for word in text:
        value=termid[word]
        inv_indx[value].append(idx+1)

In [197]:
inv_indxl=dict(inv_indx)

In [191]:
for key in list(inv_indxl):
    if type(key) is not str:
        try:
            nv_indxl[str(key)] = inv_indxl[key]
        except:
            try:
                inv_indxl[repr(key)] = inv_indxl[key]
            except:
                pass
            del inv_indxl[key]

In [192]:
with open('inv_indx.json', 'w') as fp:
    json.dump(inv_indxl, fp)

In [193]:
with open('incindx.json', 'r') as fp:
    inv_indx = json.load(fp)
inv_indx={int(key):inv_indx[key] for key in inv_indx}

#### Step 3.1.2 :
######  Querying the Search Engine

In [173]:
q1=input()

kitchen bedroom


In [174]:
# Using built-in Library from NLTK for Stop Words
stemmer = SnowballStemmer("english")
df=pd.DataFrame(columns=['title','description'])
ps = PorterStemmer()
stop = stopwords.words('english')

In [175]:
# Filtering the Stopwords,Punctuations,Accents, Stem words from the Search Query

stopword=lambda l1: ' '.join([word for word in l1.split() if word not in (stop)])
punctuation= lambda x:''.join([i for i in x if i not in string.punctuation])
q1=stopword(q1)
q1=punctuation(q1)
q1=stem_sentences(q1)
q1=remove_punctuations(q1)
q1=remove_accents(q1)

In [163]:
# Fetching the TermId's from the dictionary for the terms in Search Query

tid1=[]
for l in (q1.split()):
    if l in termid:
        value=termid[l]
        tid1.append(value)

In [187]:
# Fetching all the documentID's that contains the Terms in the Search Query from the Inverted Index

docid1=[]
for t in tid1:
    value=inv_indx[t]
    docid1.append(set(value))
u = set.intersection(*docid1)

##### Search Results for the User Query

In [49]:
# Populating the Search Results into a DataFrame with Title , Description , City & Url 

p=pd.DataFrame()
l = ['average_rate_per_night', 'bedrooms_count', 'city', 'date_of_listing', 'description' , 'latitude', 'longitude','title','url']
for b in u:
    d=pd.read_csv("/home/atefe/Desktop/airbnb-property-data-from-texas4/doc_"+str(b)+".tsv",names=l,delimiter="\t")
    p=p.append(d[['description','title','city','url']])

In [50]:
p

,description,title,city,url
9733,Our cool and cozy condo is on the beach and wa...,Cozy 2 bedroom condo on the beach,South Padre Island,https://www.airbnb.com/rooms/17232476?location...
15885,A centrally-located vacation oceanfront home r...,Beautiful Oceanfront Condo with Private Balcony,Galveston,https://www.airbnb.com/rooms/18447939?location...
13327,Spacious rare custom Oceanfront (streetside)/O...,Oceanfront/view home Mustard Seed #surfsidetx,Surfside Beach,https://www.airbnb.com/rooms/12004646?location...
15380,SLEEPS 60 PEOPLE 12 bedrooms 8 bathrooms. Gr...,2 Close to Sea - BEACHFRONT home,Galveston,https://www.airbnb.com/rooms/9813375?location=...
13846,Relaxing Oceanview 2 bedroom 1 bath Beachouse....,"""""""Just Beachy """"""",Surfside Beach,https://www.airbnb.com/rooms/17476997?location...
1047,Great location- fish from the dock or walk to ...,North Padre Waterfront Condo Boat/Fishing Dock...,Corpus Christi,https://www.airbnb.com/rooms/4053174?location=...
2085,This is a one bedroom apartment across the str...,Downtown Corpus near the water!,Corpus Christi,https://www.airbnb.com/rooms/16888519?location...
16428,Our place is great for couples families (with...,Copano Bay Retreat,Bayside,https://www.airbnb.com/rooms/13589536?location...
554,4 Bedrooms 3.5 Baths Sleeps 10\nThis cottage b...,DANCING STARFISH COTTAGE** STEPS AWAY FROM POOL!,Port Aransas,https://www.airbnb.com/rooms/17134486?location...
15918,Beautiful well-maintained home with 3 Bedrooms...,Chic Modern Galveston Cottage Near Pleasure Pier,Galveston,https://www.airbnb.com/rooms/16124268?location...


### Step 3.2 :
#### Conjunctive query & Ranking score

#### Step 3.2.1 :
##### Creating a New Inverted Index

In [198]:
new_invindx={}
for k in termid:
    t=termid[k]
    l=inv_indx[t]
    new_invindx[t]=[]
    for i in l:
        new_invindx[t].append((i,X[((i-1),t)]))

In [199]:
new_invindx={str(key):new_invindx[key] for key in inv_indx}

In [200]:
with open('new_invindx.json', 'w') as fp:
    json.dump(new_invindx, fp)

In [201]:
with open('new_invindx.json', 'r') as fp:
    new_invindx = json.load(fp)
new_invindx={int(key):new_invindx[key] for key in new_invindx}

#### Step 3.2.2 :
######  Querying the Search Engine

In [202]:
# Fetching the TermId's from the dictionary for the terms in Search Query

tid=[]
for l in (q1.split()):
    if l in termid:
        value=termid[l]
        tid.append(value)

In [203]:
# Fetching all the documentID's that contains the Terms in the Search Query from the Inverted Index
docid=[]
for t in tid:
    value=inv_indx[t]
    docid.append(set(value))
u = set.intersection(*docid)

In [144]:
# Search Results with Document ID's 


In [204]:
Q=q1.split(" ")

In [205]:
Q=vectorize.transform([q1])

In [206]:
M=Q.tocoo()
d1 = dict(zip(M.col, M.data))

In [207]:
df1=pd.read_csv("new_backup_title_desc.csv")

In [208]:
dn = df1['title']+' '+df1['description']
d2 = pd.DataFrame(dn)
d2.columns = ["words"]

In [209]:
w=d2['words'].str.split(' ', expand=True).stack().value_counts()
wordlist=list(w.index)

In [210]:
# Calculating the Cosine Similarity Ranked by their Score
"""in the following script we calculate the cosine similarity between query and documents that contain all the 
                            words of the query"""
""" based on the formula of cosine similarity in the following script we  should have two vectors of the same size
    therefore we assign a key(here is termid) in the dictionary with the value 0 if the term is not in the query
    but it is in the document"""
precosl={}
finalcosl={}
for d in list(u):
    temp=d1
    g=d2.loc[(d-1),'words']
    g=g.split(" ")
    for i in g:
        if i in termid:
            k=termid[i]
            tl=new_invindx[k]
            for r in tl:
                if r[0]==d:
                    precosl[termid[i]]=r[1]
    for m in precosl:
        if m in temp:
            continue
        else:
            temp[m]=0
    for m in temp:
        if m in precosl:
            continue
        else:
            precosl[m]=0
    doc_d= pd.Series(precosl)
    query_d = pd.Series(temp)
    finalcosl[d]=(1 - cosine(doc_d, query_d))

In [211]:
finalcosl

{16384: 0.088618959233774275,
 8193: 0.11442399147816573,
 2: 0.053213185307156796,
 8199: 0.037271304094339097,
 11: 0.07570488625930949,
 8208: 0.047443953047549337,
 8213: 0.032414364972816911,
 24: 0.045527161070617272,
 16413: 0.043382484524732479,
 8225: 0.035652567130620705,
 8229: 0.036255395582012739,
 8232: 0.037652861590505515,
 8233: 0.028151855953370108,
 16424: 0.021106838850251441,
 8241: 0.05543554557650987,
 16433: 0.027120602991546949,
 8244: 0.027390912930748357,
 53: 0.043978244561072022,
 16436: 0.028233379648937129,
 8247: 0.041570133185726665,
 58: 0.019749268825557609,
 8251: 0.025959439240560878,
 8253: 0.021478956759948931,
 16446: 0.021886614733872545,
 16448: 0.023059921524268279,
 65: 0.057561448757422862,
 8257: 0.024201842318667288,
 8270: 0.020358739630731537,
 16464: 0.034375610427092917,
 8284: 0.010802008157685172,
 8286: 0.031430670301165864,
 8292: 0.019843013629411099,
 16484: 0.032762525028915079,
 8294: 0.058127756203016134,
 16488: 0.02024016994

##### Search Results of document ID's ranked with their Cosine Similarity 

In [212]:
p1=pd.DataFrame()
l = ['average_rate_per_night', 'bedrooms_count', 'city', 'date_of_listing', 'description' , 'latitude', 'longitude','title','url']
for b in u:
    d=pd.read_csv("/home/atefe/Desktop/airbnb-property-data-from-texas5/doc_"+str(b)+".tsv",names=l,delimiter="\t")
    d['Cosine-Similiarty']=finalcosl[b]
    p1=p1.append(d[['description','title','city','url','Cosine-Similiarty']])

In [213]:
p1.sort_values(by=['Cosine-Similiarty'],ascending=False, kind='heapsort')

,description,title,city,url,Cosine-Similiarty
0,spacious privat master bedroom nd floor sqt s...,spacious privat suit center plano wifi,Plano,https://www.airbnb.com/rooms/19213591?location...,0.114424
0,central locat nd floor garag apt w privat entr...,the treehous apart rosedal,Austin,https://www.airbnb.com/rooms/18445745?location...,0.088619
0,my cool comfort bedroom apart feel like home i...,cozi bedroombathroom pool,Irving,https://www.airbnb.com/rooms/7276294?location=...,0.075705
0,my beauti remodel hous close airport mile irv...,updat modern hous,Irving,https://www.airbnb.com/rooms/16225510?location...,0.058128
0,you will love it safeti conveni comfort costef...,texa medic center bedroom bathroom room,Houston,https://www.airbnb.com/rooms/17341645?location...,0.057561
0,charm rock cottag guadalup river this precious...,casa bella on the river,Kerrville,https://www.airbnb.com/rooms/9133638?location=...,0.055436
0,stylish fulli remodel home upscal nw alamo he...,uniqu locat alamo height design inspir,San Antonio,https://www.airbnb.com/rooms/17481455?location...,0.053213
0,my hous close lakelin mall highway toll road r...,love queen bedroom nw austin,Austin,https://www.airbnb.com/rooms/14821566?location...,0.047444
0,my place close kati mill mall shop lacentera m...,luxuri master bedroom safe quit neibourhood,Katy,https://www.airbnb.com/rooms/15860117?location...,0.045527
0,the apart master queen bedroom ensuit bath one...,zanzenberg storyvill,Center Point,https://www.airbnb.com/rooms/10617388?location...,0.044293


### Step 4 :
##### Defining a new score !

In [66]:
df1=pd.read_csv("/home/atefe/Desktop/airbnb-property-data-from-texas/Airbnb_Texas_Rentalsd.csv")

In [68]:
#l = ['average_rate_per_night', 'bedrooms_count', 'city', 'date_of_listing', 'description' , 'latitude', 'longitude','title','url']
#Df=pd.DataFrame(columns=l)
#for i in range(1,18255):
 #   ds=pd.read_csv("/home/atefe/Desktop/airbnb-property-data-from-texas5/doc_"+str(i)+".tsv",delimiter="\t",names=l)
#  Df=Df.append(ds)

In [ ]:
Df[['bedrooms_count','average_rate_per_night']]=Df[['bedrooms_count','average_rate_per_night']].fillna(0)
Df.to_csv("Defining_newscore.csv"

In [70]:
Df=pd.read_csv("Defining_newscore.csv")

In [71]:
#Cleaning the data and extracting meaningful information to define New Score based on City,No of Bedrooms & Average rate/Night
#we remove the $ sign in average_rate_per_night and convert it to float
Df['average_rate_per_night']=Df['average_rate_per_night'].apply(lambda x : int(x.replace('$','')))
Df['bedrooms_count']=Df['bedrooms_count'].apply(lambda x : str(x))
#we convert the bedroom to float and then we replace the Studio with 1
Df['bedrooms_count']=Df['bedrooms_count'].apply(lambda x : float(x.replace('Studio','1')))

######  Querying the Search Engine with City , Number of BedRooms & Average rate / Night 

In [72]:
c=input('please Enter city: ')

please Enter city: Port Aransas


In [73]:
b=input('please Enter the number of bedrooms? ')

please Enter the number of bedrooms? 4


In [74]:
if b !="Studio":
    b=int(b)
if b=="Studio":
    b=1

In [75]:
r=input('please Enter the number of average_rate_per_night (in dollar)? ')
r=int(r.replace('$',''))

please Enter the number of average_rate_per_night (in dollar)? 70


In [76]:
"""u is the intersection of documents that contains the word from previous part in other words it contains all 
the documents that contains all the words in the query"""
u = [x-1 for x in u]
S=Df.ix[u]

In [77]:
S=S.reset_index(drop=True)

In [78]:
"""Using Builtin library Geopy & Nominatim in order to find the distance between the city that is in the query
and the city in the documents"""

geolocator = Nominatim(user_agent='myapplication')
try:
    location = geolocator.geocode(c,timeout=None)
    lan=location.latitude
    lon=location.longitude
except GeocoderTimedOut as e:
    lan=100
Y=(lan,lon)

In [132]:
"""Is the Longtitude and latitude of the city in the query"""
Y

(27.8332175, -97.0618324)

In [79]:
"""the w vector contains the score , 0.1 is for bedroom , 0.3 is for the price and 0.6 for the city and 0.4 for
   the distance of the city and other cities """
"""the document will get the score of bedroom if the bedroom is one point less or more than the bedroom entered"""
"""the document will get the score of price if the price is 20 dollar  less or more than the price that
entered by the user"""
"""the document will get the score of city if the city is excatly as the user entered"""
"""if there isn't any city the same as the user entered we calculate the distance and if we find a house that is 
20 km away the location that the user enterd this document will gain the score of distance """
w=[0.1,0.3,0.6,0.4]
for i in range(len(S)):
    m=[]
    
    if b-1<=S.loc[i,'bedrooms_count']<=b+1:
        m.append(1)
    else:
        m.append(0)
    if r-20<=S.loc[i,'average_rate_per_night']<=r+10:
        m.append(1)
    else:
        m.append(0)
    if c==S.loc[i,'city']:
        m.append(1)
        m.append(0)
    else:
        m.append(0)
        X=(S.loc[i,'latitude'],S.loc[i,'longitude'])
        Dist=great_circle(X,Y).km
        if Dist<=10:
            m.append(1)
        else:
            m.append(0)
    DotProduct = np.dot(m,w)
    S.loc[i,'Score']=DotProduct

#### Search results with a New Score

In [80]:
S.sort_values(by=['Score'],ascending=False, kind='heapsort')

,Unnamed: 0,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url,Score
141,505,469,4.0,Port Aransas,May 2016,larg multilevel home bedroom bath two larg c...,27.813243,-97.071795,spacious beach hous,https://www.airbnb.com/rooms/13012107?location...,0.7
8,552,375,4.0,Port Aransas,July 2016,bedroom bath sleep nthis cottag bring beach ...,27.812983,-97.077364,danc starfish cottag step away from pool,https://www.airbnb.com/rooms/17134486?location...,0.7
125,14780,200,3.0,Port Aransas,July 2015,this larg openconcept beach hous locat one hig...,27.831177,-97.059427,openconcept beach hous old town,https://www.airbnb.com/rooms/13899003?location...,0.7
105,2412,200,3.0,Port Aransas,July 2015,this larg openconcept beach hous locat one hig...,27.831177,-97.059427,openconcept beach hous old town,https://www.airbnb.com/rooms/13899003?location...,0.7
101,15711,352,3.0,Port Aransas,October 2016,our place close beach shop restaur citi center...,27.813637,-97.075778,bed bath buccan retreat sleep,https://www.airbnb.com/rooms/15494997?location...,0.7
48,2715,250,3.0,Port Aransas,June 2016,super cute singl stori great multipl famili co...,27.831325,-97.062039,charm stori heart port a,https://www.airbnb.com/rooms/18410046?location...,0.7
91,16697,250,3.0,Port Aransas,June 2016,super cute singl stori great multipl famili co...,27.831325,-97.062039,charm stori heart port a,https://www.airbnb.com/rooms/18410046?location...,0.7
32,17517,375,4.0,Port Aransas,July 2016,bedroom bath sleep nthis cottag bring beach ...,27.812983,-97.077364,danc starfish cottag step away from pool,https://www.airbnb.com/rooms/17134486?location...,0.7
26,2142,650,4.0,Port Aransas,October 2015,brand new custom built home featur larg open k...,27.804527,-97.077024,gone coastal new home close beach town,https://www.airbnb.com/rooms/12360217?location...,0.7
83,3356,352,3.0,Port Aransas,October 2016,our place close beach shop restaur citi center...,27.813637,-97.075778,bed bath buccan retreat sleep,https://www.airbnb.com/rooms/15494997?location...,0.7


### Bonus Step : 
##### Making a nice visualization  Listing  all the search results within a given radius !

In [2]:
# Loading Data and creating a new Column with Coordinates information for a particular listing

df=pd.read_csv("Airbnb_Texas_Rentals.csv")

df["coordinates"]= df.apply(lambda x: [x.latitude, x.longitude], axis=1)
                                                                        

In [3]:
# Cleaning the data for missing values and Resetting the Index

df["coordinates"]= df["coordinates"].apply(lambda v: pd.Series(v).dropna().values)  
df["coordinates"]=df["coordinates"][df["coordinates"].apply (len) > 0]
df= df.loc[df.coordinates.notnull()]
df.index = range(len(df.index))


In [4]:
df.head()

,Unnamed: 0,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url,coordinates
0,1,$27,2,Humble,May 2016,Welcome to stay in private room with queen bed...,30.020138,-95.293996,2 Private rooms/bathroom 10min from IAH airport,https://www.airbnb.com/rooms/18520444?location...,"[30.0201379199512, -95.29399600425128]"
1,2,$149,4,San Antonio,November 2010,"Stylish, fully remodeled home in upscale NW – ...",29.503068,-98.447688,Unique Location! Alamo Heights - Designer Insp...,https://www.airbnb.com/rooms/17481455?location...,"[29.503067675606104, -98.4476879378504]"
2,3,$59,1,Houston,January 2017,'River house on island close to the city' \nA ...,29.829352,-95.081549,River house near the city,https://www.airbnb.com/rooms/16926307?location...,"[29.829352227214898, -95.0815494887563]"
3,4,$60,1,Bryan,February 2016,Private bedroom in a cute little home situated...,30.637304,-96.337846,Private Room Close to Campus,https://www.airbnb.com/rooms/11839729?location...,"[30.6373042787676, -96.33784597296308]"
4,5,$75,2,Fort Worth,February 2017,Welcome to our original 1920's home. We recent...,32.747097,-97.286434,The Porch,https://www.airbnb.com/rooms/17325114?location...,"[32.7470973543511, -97.2864343970125]"


#### Querying the Search Engine to display Properties for a Location

In [5]:
#Using Libraries to get the coordinates of a by Searching for a particular building, street or a desired place(Manual Text Input)

geolocator= Nominatim()
address=input(" Type Name of building, Street or Place : ")
location= geolocator.geocode(address)
cen_alt=[]
cen_alt.append(location.latitude)
cen_alt.append(location.longitude)

print(cen_alt)
rad=float(input(" Type Radius in metres: "))

C:\Users\VICKY\Anaconda3\lib\site-packages\geopy\geocoders\osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


 Type Name of building, Street or Place : Humble
[29.9988312, -95.2621553]
 Type Radius in metres: 10000


In [6]:
# Creating a Circle within the given radius for search criteria


m = folium.Map(location=cen_alt, zoom_start=12)
circle = folium.Circle(
    radius=rad, # circle radius
    location=cen_alt, # circle center
    color='crimson',
    fill=True, 
).add_to(m) 


            
tooltip = 'Click me!'
mark=folium.Marker(cen_alt, popup= address, tooltip=tooltip, icon=folium.Icon(color="red")).add_to(m)

In [7]:
m

#### Displaying the Houses on Airbnb and directing the User to the landing page of advertisement !

In [8]:
# Displaying Total Houses inside the circle for Search Criteria 
# Initialising the Count and Display with Zero

dis=0
count=0
j=0
tooltip = 'Click me!'
for i in df["coordinates"]:
    dis=(geopy.distance.vincenty(i, cen_alt).km)*1000
    if dis<=rad:
        url=df["url"].loc[j]
        price= df["average_rate_per_night"].loc[j]
        popup = folium.Popup('<a href="' + url + '"target="_blank">' + price + '</a>')
        folium.Marker(i, popup=popup, tooltip=tooltip).add_to(m)
        count+=1
        
    j+=1

In [9]:
m.save("Humble_searchResults.html")

In [10]:
 # Displaying the Total Count of Houses found within the given Radius

print(count,"Total Houses found for ",address)

m

138 Total Houses found for  Humble
